In [3]:
#Summarise camera data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df=pd.read_csv(r'camera_named.csv')
df.describe()

,bid_id,item_id,buy_it_now,reserve_price,seller_feedback,bid_price,buyer_feedback
count,44110.000000,4.411000e+04,44110.00000,44110.000000,44110.000000,44110.000000,44110.000000
mean,189757.270052,3.235042e+08,454.91377,50.813888,21.591952,313.403129,1.005078
std,20844.384520,1.417769e+07,222.48069,120.139940,10.909127,217.947058,0.196021
min,149685.000000,2.952551e+08,1.00000,0.010000,1.000000,0.010000,0.000000
25%,173642.250000,3.124101e+08,316.11000,1.000000,15.000000,100.000000,1.000000
50%,192209.500000,3.220564e+08,480.00000,1.500000,21.000000,310.000000,1.000000
75%,208296.750000,3.346011e+08,620.00000,22.990000,27.000000,477.695000,1.000000
max,220273.000000,3.519200e+08,1250.00000,1000.000000,96.000000,1250.000000,30.000000


In [12]:
items = df.groupby(['item_id'])['bid_price'].count().reset_index(name='Count').sort_values(['Count'], ascending=False)
#items.sort_values('item_id', ascending=False)
print(items)

        item_id  Count
4067  346035866     68
1554  316509884     60
908   310799207     54
4119  346991188     48
1966  319756646     46
...         ...    ...
1371  314711789      1
3846  342544657      1
552   306147586      1
553   306152307      1
479   304886627      1

[4388 rows x 2 columns]
